# Import Libraries

In [2]:
#pip install bs4
#import bs4
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen, HTTPError
import re
import pandas as pd

from datetime import datetime,timedelta

import time
from http.client import IncompleteRead
from pprint import pprint
import requests

from urllib.error import HTTPError

# Read csv file

In [3]:
#retrieve historical data throug APIs 
df1 = pd.read_excel("D:\\Research Project\\Datasets\\Flood intensity\\data url\\xlxs files\\date1_url.xlsx")
df1 ####4926

,Unnamed: 0,ID,GlideNumber,Country,OtherCountry,long,lat,Area,Began,Ended,Validation,Dead,Displaced,MainCause,Severity,coordinate url,began date url
0,0,1,0,Algeria,0,5.23026,35.814200,92615.67,1985-01-01,1985-01-05,News,26,3000,Heavy rain,1.0,"https://www.wunderground.com/weather/35.8142,5...",https://www.wunderground.com/history/daily/dz/...
1,1,2,0,Brazil,0,-45.34890,-18.711100,678498.82,1985-01-15,1985-02-02,News,229,80000,Heavy rain,2.0,"https://www.wunderground.com/weather/-18.7111,...",https://www.wunderground.com/history/daily/br/...
2,2,3,0,Phillipines,0,122.97400,10.020700,12846.03,1985-01-20,1985-01-21,News,43,444,Torrential rain,1.0,"https://www.wunderground.com/weather/10.0207,1...",https://www.wunderground.com/history/daily/ph/...
3,3,4,0,Indonesia,0,124.60600,1.014890,16542.12,1985-02-04,1985-02-18,News,21,300,Torrential rain,1.0,"https://www.wunderground.com/weather/1.01489,1...",https://www.wunderground.com/history/daily/id/...
4,4,5,0,Mozambique,0,32.34910,-25.869300,20082.21,1985-02-09,1985-02-11,News,19,0,Heavy rain,2.0,"https://www.wunderground.com/weather/-25.8693,...",https://www.wunderground.com/history/daily/mz/...
5,5,6,0,Comoros islands,0,43.36000,-11.651600,1035.61,1985-02-16,1985-02-28,News,2,35000,Tropical cyclone,1.0,"https://www.wunderground.com/weather/-11.6516,...",https://www.wunderground.com/history/daily/km/...
6,6,7,0,New Zealand,0,175.73400,-37.230500,7871.37,1985-02-17,1985-02-18,News,4,200,Heavy rain,1.0,"https://www.wunderground.com/weather/-37.2305,...",https://www.wunderground.com/history/daily/nz/...
7,7,8,0,Indonesia,0,108.14000,-7.040080,77091.11,1985-02-19,1985-02-23,News,10,2000,Heavy rain,1.0,"https://www.wunderground.com/weather/-7.04008,...",https://www.wunderground.com/history/daily/id/...
8,8,9,0,USA,0,-85.17420,40.669100,210527.96,1985-02-22,1985-03-01,News,7,2250,Rain and snowmelt,2.0,"https://www.wunderground.com/weather/40.6691,-...",https://www.wunderground.com/history/daily/us/...
9,9,10,0,Bolivia,0,-63.28870,-21.224400,69706.89,1985-02-25,1985-02-28,News,2,1000,Heavy Rain,1.0,"https://www.wunderground.com/weather/-21.2244,...",https://www.wunderground.com/history/daily/bo/...


# weatherunderground API to retrive weather data for Day before flood Day (Day minus 1)

In [4]:
#retrieve historical data from weatherunderground API 
for i, row in df1.iterrows():
    
    URL = df1.loc[i,"began date url"]
    
    start_date = df1.loc[i,"Began"]
    start_date = ((start_date).date())
    start_date=start_date-timedelta(days=1)# 1 days before
    start_date=str(start_date)
    start_date = start_date.replace('-', '')
    print("URL :",URL)
    try:
        print("i :",i)
        req = Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')
        value1 = soup.find('div', {'class':'station-name ng-star-inserted'})#this is to get the city code like br/cerro-azul/ICURITIB26

        if value1 is None:
            j=i
        else:
            value2=value1.find('span',{'class':"station-id"})
            value3 = (value2.contents[0][1:-1])
            result_list = re.sub(r"^.+?(?=daily)", "", URL) #get the new values to add to the new_url
            string_in_slashes = result_list.split('/')[1].strip() # to get the country code from URL
            string_in_slashes.upper()
            new_url = 'https://api.weather.com/v1/location/' +value3+':9:'+string_in_slashes.upper()+'/observations/historical.json?apiKey=6532d6454b8aa370768e63d6ba5a832e&units=e&startDate='+start_date
            try:
                
                res=requests.get(new_url)
                print("opened new url res:",res)
                w_u = res.json()
                df1.loc[i,"Time"] = w_u['observations'][0]['expire_time_gmt']

                df1.loc[i,"Temperature"] = w_u['observations'][0]['temp']
                df1.loc[i,"Dew Point"] = w_u['observations'][0]['dewPt']
                df1.loc[i,"Humidity"] = w_u['observations'][0]['rh']

                df1.loc[i,"Wind"] = w_u['observations'][0]['wdir_cardinal']

                df1.loc[i,"Wind Speed"] = w_u['observations'][0]['wspd']
                df1.loc[i,"Wind Gust"] = w_u['observations'][0]['gust']
                df1.loc[i,"Pressure"] = w_u['observations'][0]['pressure']
                df1.loc[i,"Precipitation"] = w_u['observations'][0]['precip_total']

                df1.loc[i,"Condition"] = w_u['observations'][0]['wx_phrase']

            except KeyError:
                continue

            except IncompleteRead:
                # Oh well, reconnect and keep trucking
                continue 
            except NameError:
                continue

    except HTTPError as err:
        if err.code == 404:
            continue
        else:
            continue
    except UnicodeEncodeError:
        continue
    except ValueError:
        continue
    except (KeyError, IncompleteRead, NameError, urllib.error.HTTPError, AttributeError, UnicodeEncodeError, TypeError, ValueError, gaierror, URLError ):
        print("inside except")
        continue

#     except IncompleteRead:
#         print("inside IncompleteRead")
#         # Oh well, reconnect and keep trucking
#         continue 
#     except NameError:
#         print("inside NameError")
#         continue
#     except HTTPError:
#         print("inside HTTPError")
#         continue

    finally:
        print("inside finally")
        k=i

    if i%50 ==0:
        print(f"{i} rows iterated")
        # Wait for 5 seconds
        time.sleep(5)

        





URL : https://www.wunderground.com/history/daily/dz/rasfa/date/1984-12-31/
i : 0
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/br/morada-nova-de-minas/date/1985-01-14/
i : 1
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/ph/himamaylan/date/1985-01-19/
i : 2
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/id/minahasa-tenggara/date/1985-02-03/
i : 3
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/mz/date/1985-02-08/
i : 4
inside finally
URL : https://www.wunderground.com/history/daily/km/IMAYOT3/date/1985-02-15/
i : 5
inside finally
URL : https://www.wunderground.com/history/daily/nz/whangamata/IWHANG48/date/1985-02-16/
i : 6
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/id/majalengka/date/1985-02-18/
i : 7
opened new url 

opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/us/tx/zephyr/KTXEARLY3/date/1985-10-18/
i : 61
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/ph/dupax-del-sur/date/1985-10-18/
i : 62
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/us/la/lafitte/KLABARAT2/date/1985-10-25/
i : 63
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/us/va/fort-valley/KVASEVEN1/date/1985-10-31/
i : 64
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/jm/may-pen/date/1985-11-18/
i : 65
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/ve/guaicaipuro/date/1985-12-01/
i : 66
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/tt/st.-mary/ICHAGUAN6/date/1985

opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/ec/el-chaco/date/1987-03-04/
i : 123
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/pe/san-antonio/date/1987-03-08/
i : 124
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/af/date/1987-03-15/
i : 125
inside finally
URL : https://www.wunderground.com/history/daily/us/me/pittsfield/KMEPLYMO3/date/1987-03-30/
i : 126
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/ec/naranjal/IGUAYAQU17/date/1987-04-02/
i : 127
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/rs/Å¡abac/date/1987-05-06/
i : 128
inside finally
URL : https://www.wunderground.com/history/daily/au/INIMBI2/date/1987-05-08/
i : 129
inside finally
URL : https://www.wunderground.com/history/daily/pl/klÄ™czany/IWOJEWDZ51/dat

opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/af/IKABUL7/date/1988-06-12/
i : 184
inside finally
URL : https://www.wunderground.com/history/daily/cn/chongqing/date/1988-06-09/
i : 185
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/tr/ankara/date/1988-06-11/
i : 186
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/bd/jessore/date/1988-06-13/
i : 187
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/cn/shangrao/date/1988-06-10/
i : 188
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/in/seepa/date/1988-06-20/
i : 189
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/ua/date/1988-06-19/
i : 190
inside finally
URL : https://www.wunderground.com/history/daily/cn/leshan/date/1988-06

inside finally
URL : https://www.wunderground.com/history/daily/np/syalagadi/date/1988-09-19/
i : 249
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/th/nong-bua-daeng/date/1988-09-17/
i : 250
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/cn/quanzhou/date/1988-09-19/
i : 251
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/in/moga/date/1988-09-20/
i : 252
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/tw/xiulin/ITAIWANN2/date/1988-09-20/
i : 253
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/in/mumbai/date/1988-10-02/
i : 254
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/fr/nÃ®mes/INMES90/date/1988-10-02/
i : 255
inside finally
URL : https://www.wunderground.com/histor

...

opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/in/madikeri/date/2018-07-08/
i : 4653
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/sk/pochabany/IBNOVC1/date/2018-07-17/
i : 4654
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/la/sanxay-district/date/2018-07-21/
i : 4655
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/fr/savournon/date/2018-08-08/
i : 4656
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/dz/tazrouk/date/2018-08-01/
i : 4657
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/ca/whitby/IOSHAWA3/date/2018-08-06/
i : 4658
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/co/puerto-colombia/date/2018-07-25/
i : 46

opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/ph/davao-city/date/2019-01-25/
i : 4715
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/au/IQLDCOOY2/date/2019-01-25/
i : 4716
inside finally
URL : https://www.wunderground.com/history/daily/br/brumadinho/date/2019-01-24/
i : 4717
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/es/belmonte-de-miranda/IGRADO7/date/2019-01-21/
i : 4718
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/ec/mera/date/2019-02-08/
i : 4719
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/gr/zaros/date/2019-02-11/
i : 4720
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/zw/IHARARE11/date/2019-02-11/
i : 4721
inside finally
URL : https://www.wunderground.

opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/in/rayagada/date/2019-08-04/
i : 4782
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/sd/date/2019-08-06/
i : 4783
inside finally
URL : https://www.wunderground.com/history/daily/mr/date/2019-08-24/
i : 4784
inside finally
URL : https://www.wunderground.com/history/daily/ma/touizgui/date/2019-08-19/
i : 4785
opened new url res: <Response [400]>
inside finally
URL : https://www.wunderground.com/history/daily/jp/tamana-shi/date/2019-08-27/
i : 4786
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/bs/date/2019-08-31/
i : 4787
inside finally
URL : https://www.wunderground.com/history/daily/dz/ISANTA155/date/2019-08-31/
i : 4788
inside finally
URL : https://www.wunderground.com/history/daily/cf/date/2019-08-18/
i : 4789
inside finally
URL : https://www.wunderground.com/history/daily/ru/ko

opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/ug/date/2019-12-03/
i : 4850
inside finally
4850 rows iterated
URL : https://www.wunderground.com/history/daily/pe/moyobamba/date/2019-12-14/
i : 4851
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/us/tn/chapel-hill/KTNCHAPE4/date/2019-12-27/
i : 4852
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/co/lejanÃ­as/date/2019-12-24/
i : 4853
inside finally
URL : https://www.wunderground.com/history/daily/pe/codo-del-pozuzo-district/date/2019-12-05/
i : 4854
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/co/mercaderes/date/2019-12-05/
i : 4855
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/id/date/2019-12-31/
i : 4856
inside finally
URL : https://www.wunderground.com/history/daily/

opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/hn/manto/date/2020-05-25/
i : 4919
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/om/thumrait/date/2020-05-26/
i : 4920
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/cf/date/2020-05-23/
i : 4921
inside finally
URL : https://www.wunderground.com/history/daily/ug/date/2020-05-20/
i : 4922
inside finally
URL : https://www.wunderground.com/history/daily/id/kutai-kartanegara/date/2020-05-21/
i : 4923
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/in/kannad/date/2020-06-01/
i : 4924
opened new url res: <Response [200]>
inside finally
URL : https://www.wunderground.com/history/daily/gt/flores/ISPANI4/date/2020-06-01/
i : 4925
opened new url res: <Response [400]>
inside finally


Check for Null Values

In [5]:
#successfully iterated all rows throug wu API; hence check for null rows
df1_is_null = df1[df1['Time'].isnull()]
print(len(df1_is_null))####2956 rows
df1_not_null = df1[df1['Time'].notnull()]
print(len(df1_not_null))####1970 rows 

2956
1970


Export Not Null Data as backup

In [6]:
#export nul null rows
df1_not_null.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 1 file\dayminus4 1970 rows processed wu.csv')

In [7]:
#df1_is_null for ow API
df1_is_null_for_ow = df1_is_null[df1_is_null['Began'] < '01-07-2008'] ### need to add these details for ow API
print(len(df1_is_null_for_ow)) ########2256 


2256


Select data from worldweather API

In [8]:
#retrieve historical data from 1st July 2008 from worldweatheronline API 
df2 = df1_is_null[df1_is_null['Began'] >= '01-07-2008']
print(len(df2)) ### 700

700


In [31]:
#exporting df2 as csv to run in another system as daily limit is over --- however it wasnt fruitful
#df2.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 4 file\dayminus4 692 rows to be processed for wwo.csv')

# Data retrieve via worldweather API

In [11]:
import urllib
import urllib.parse
import json
import pandas as pd
from datetime import datetime
import os

##################################
# main function to retrive the data by location list
def retrieve_hist_data(api_key, location_list, start_date):
    
    for location in location_list:
        
        start_d= str(start_date)
        end_d = str(start_date)
        frequency=1
        url_page = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=' + api_key + '&q=' + location + '&format=json&date=' + start_d + '&enddate=' + end_d + '&tp=' + str(
                frequency)
        print("url_page :",url_page)
        json_page = urllib.request.urlopen(url_page, timeout=10)
        
        json_data = json.loads(json_page.read().decode())
        

        print("data collection for row :",i)
        data = json_data['data']['weather'][0]['date']
        data1 = json_data['data']['weather'][0]['avgtempF']
        data2 = json_data['data']['weather'][0]['hourly'][0]["DewPointF"]
        data3 = json_data['data']['weather'][0]['hourly'][0]["humidity"]
        data4 = json_data['data']['weather'][0]['hourly'][0]["winddir16Point"]
        data5 = json_data['data']['weather'][0]['hourly'][0]["windspeedMiles"]
        data6 = json_data['data']['weather'][0]['hourly'][0]["WindGustMiles"]
        data7 = json_data['data']['weather'][0]['hourly'][0]["pressureInches"]
        data8 = json_data['data']['weather'][0]['hourly'][0]["precipInches"]
        data9 = json_data['data']['weather'][0]['hourly'][0]["weatherDesc"][0]["value"]
        
    
    return (data, data1, data2, data3, data4, data5, data6, data7, data8, data9)
##################################



Function to retrive data from worldweather API

In [14]:
for i, row in df2.iterrows():
    #internet went off, so retrying afte 4546th row
    if i >= 4546:
        start_date = df2.loc[i,"Began"]
        start_date = ((start_date).date())
        start_date=start_date-timedelta(days=1)#1 days before
        #api_key = '365f0b99b7f2410fbb8111748201107'
        #api_key = 'dfd495982ade45cfb30183830201507'
        api_key = '064d3cdae3804e168e501356201107'
        location_list = [str(df2.loc[i,"lat"])+','+str(df2.loc[i,"long"])]
        print("row number :",i)
        try :
            data, data1, data2, data3, data4, data5, data6, data7, data8, data9 = retrieve_hist_data(api_key,location_list,start_date)
            df2.loc[i,"Time"]=data
            df2.loc[i,"Temperature"]=data1
            df2.loc[i,"Dew Point"]=data2
            df2.loc[i,"Humidity"]=data3
            df2.loc[i,"Wind"]=data4
            df2.loc[i,"Wind Speed"]=data5
            df2.loc[i,"Wind Gust"]=data6
            df2.loc[i,"Pressure"]=data7
            df2.loc[i,"Precipitation"]=data8
            df2.loc[i,"Condition"]=data9


        except KeyError:
            continue

        except IncompleteRead:
            # Oh well, reconnect and keep trucking
            continue 
        except NameError:
            continue
        except HTTPError:
            continue


        if i%5 ==0:
            print(f"{i} rows iterated")
            # Wait for 5 seconds
            time.sleep(5)

row number : 4546
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=-36.0305,144.772&format=json&date=2017-11-30&enddate=2017-11-30&tp=1
data collection for row : 4546
row number : 4547
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=7.76105,80.7359&format=json&date=2017-11-28&enddate=2017-11-28&tp=1
data collection for row : 4547
row number : 4550
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=-13.9097,34.1563&format=json&date=2017-12-15&enddate=2017-12-15&tp=1
data collection for row : 4550
4550 rows iterated
row number : 4551
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=24.3598,55.9677&format=json&date=2017-12-15&enddate=2017-12-15&tp=1
data collection for row : 4551
row number : 4553
url_page : http://api.worldweatheronline.com/prem

row number : 4637
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=33.3752,75.1439&format=json&date=2018-06-24&enddate=2018-06-24&tp=1
data collection for row : 4637
row number : 4639
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=42.6984,23.9001&format=json&date=2018-06-27&enddate=2018-06-27&tp=1
data collection for row : 4639
row number : 4642
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=33.3024,72.5706&format=json&date=2018-07-02&enddate=2018-07-02&tp=1
data collection for row : 4642
row number : 4644
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=19.0183,-69.7045&format=json&date=2018-07-08&enddate=2018-07-08&tp=1
data collection for row : 4644
row number : 4645
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.

row number : 4734
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=-3.14741,29.8881&format=json&date=2019-03-22&enddate=2019-03-22&tp=1
data collection for row : 4734
row number : 4737
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=-22.6847,-43.2042&format=json&date=2019-04-06&enddate=2019-04-06&tp=1
data collection for row : 4737
row number : 4739
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=39.8586,-0.076316&format=json&date=2019-04-17&enddate=2019-04-17&tp=1
data collection for row : 4739
row number : 4740
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=-26.8541,-60.5494&format=json&date=2019-04-19&enddate=2019-04-19&tp=1
data collection for row : 4740
4740 rows iterated
row number : 4741
url_page : http://api.worldweatheronline.com

4805 rows iterated
row number : 4806
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=9.77189,-72.2284&format=json&date=2019-10-04&enddate=2019-10-04&tp=1
data collection for row : 4806
row number : 4807
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=16.0814,-96.5636&format=json&date=2019-09-26&enddate=2019-09-26&tp=1
data collection for row : 4807
row number : 4810
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=6.49722,30.5066&format=json&date=2019-10-11&enddate=2019-10-11&tp=1
row number : 4811
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=-5.45658,36.8657&format=json&date=2019-10-11&enddate=2019-10-11&tp=1
data collection for row : 4811
row number : 4814
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=0

row number : 4883
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=-1.99445,29.8633&format=json&date=2020-03-01&enddate=2020-03-01&tp=1
row number : 4886
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=20.691,105.183&format=json&date=2020-03-01&enddate=2020-03-01&tp=1
row number : 4889
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=40.132,-0.101059&format=json&date=2020-03-31&enddate=2020-03-31&tp=1
row number : 4890
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=-6.92513,146.752&format=json&date=2020-03-31&enddate=2020-03-31&tp=1
row number : 4892
url_page : http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=064d3cdae3804e168e501356201107&q=33.826,65.6424&format=json&date=2020-03-20&enddate=2020-03-20&tp=1
row number : 4

Export Not Null data as backup

In [16]:
#check for number of Null rows in wwo 
df2_not_null = df2[df2['Time'].notnull()]
len(df2_not_null) #####614
df2_not_null.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 1 file\dayminus1 614 not null rows processed wwo.csv')

In [17]:
#check for number of Null rows in wwo 
df2_is_null = df2[df2['Time'].isnull()]
len(df2_is_null) #####86

86

In [18]:
#combine required null dfs for ow
all_null_df = pd.concat([df1_is_null_for_ow  ,df2_is_null], sort=False)
print(len(all_null_df))####2342

2342


# Openweather API to retrive data

In [19]:
from pprint import pprint
import requests


def weather_data(query):
    res=requests.get('http://api.openweathermap.org/data/2.5/weather?'+query+
                  '&APPID=b35975e18dc93725acb092f7272cc6b8&units=metric');
    return res.json();          


In [20]:
########change the date before running
import time
import datetime
import re

for i, row in all_null_df.iterrows(): 
    api_key = '365f0b99b7f2410fbb8111748201107'
    lat = str(all_null_df.loc[i,"lat"])
    long = str(all_null_df.loc[i,"long"])
    start = (all_null_df.loc[i,"Began"]).date()
    start=start-timedelta(days=1)### 1 days before
    newformat = start.strftime('%d-%m-%Y')
    dt = str(int(time.mktime(datetime.datetime.strptime(newformat , "%d-%m-%Y").timetuple())))
    try:
        query='&lat='+lat+'&lon='+long+'&type=hour&start='+dt
        w_data=weather_data(query)
        data = w_data['dt']
        all_null_df.loc[i,"Time"]=data
        data1 = float(w_data['main']['temp'])
        data1=round((data1 * 9/5) + 32,2)
        all_null_df.loc[i,"Temperature"]=data1
        all_null_df.loc[i,"Humidity"] = w_data['main']['humidity']
        all_null_df.loc[i,"Wind Speed"] = w_data['wind']['speed']
        data7 = float(w_data['main']['pressure'])
        data7 = round(data7 * (0.02953/1.000000573),2) # conversion of milibars into inHg
        all_null_df.loc[i,"Pressure"] = data7
        all_null_df.loc[i,"Condition"] = w_data['weather'][0]['main']
        
        
        
        
        
    except KeyError:
        continue
    if i%50 ==0:
        print(f"{i} rows iterated")
        # Wait for 5 seconds
        time.sleep(5)

0 rows iterated
50 rows iterated
100 rows iterated
150 rows iterated
250 rows iterated
300 rows iterated
350 rows iterated
400 rows iterated
450 rows iterated
500 rows iterated
550 rows iterated
750 rows iterated
800 rows iterated
850 rows iterated
900 rows iterated
950 rows iterated
1000 rows iterated
1050 rows iterated
1100 rows iterated
1200 rows iterated
1250 rows iterated
1350 rows iterated
1450 rows iterated
1550 rows iterated
1600 rows iterated
1800 rows iterated
1850 rows iterated
1900 rows iterated
1950 rows iterated
2000 rows iterated
2100 rows iterated
2150 rows iterated
2300 rows iterated
2350 rows iterated
2400 rows iterated
2450 rows iterated
2500 rows iterated
2650 rows iterated
2750 rows iterated
2850 rows iterated
2900 rows iterated
2950 rows iterated
3000 rows iterated
3200 rows iterated
3250 rows iterated
4850 rows iterated
4900 rows iterated


Check for Null values

In [21]:
###verify for null and not null
print(len(all_null_df[all_null_df['Time'].notnull()]))####2342
print(len(all_null_df[all_null_df['Time'].isnull()]))###0

2342
0


In [23]:
#export ow dataframe
all_null_df.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 1 file\dayminus1 2342 not null rows processed ow.csv')

Merge Dataframe

In [22]:
##combine all not null dfs 
all_not_null_df = pd.concat([all_null_df, df1_not_null, df2_not_null], sort=False)
print(len(all_not_null_df))


4926


Export Day minus 1 data

In [24]:
#export all data 
all_not_null_df.to_csv('D:\Research Project\Datasets\Flood intensity\data url\Day minus 1 file\dayminus1 4926 not null rows processed.csv')